# Generating names 

In [ ]:
import os
import shutil

def rename_files_with_bin(folder_path, output):
    for root, _, files in os.walk(folder_path):
        for filename in files:
            if ".tif" in filename:                
                if "bin." in filename:
                    old_path = os.path.join(root, filename)
                    new_filename = "S0_"+ filename.replace(" bin", "_K")
                    new_path = os.path.join(output, new_filename)
                    shutil.copy2(old_path, new_path)
                else:
                    old_path = os.path.join(root, filename)
                    new_filename = "S0_"+ filename[:-4]+'_M.tif'
                    new_path = os.path.join(output, new_filename)
                    shutil.copy2(old_path, new_path)
                #if '4M' in filename:
                 #   if "bin" in filename:
                  #      old_path = os.path.join(root, filename)
                   #     new_filename = "S4_"+ filename.replace(" bin", "_K")
                    #    new_path = os.path.join(output, new_filename)
                     #   shutil.copy2(old_path, new_path)
                    #else:
                     #   old_path = os.path.join(root, filename)
                      #  new_filename = "DS_"+ filename.replace("_Smoke", "F_M")
                       # new_path = os.path.join(output, new_filename)
                        #shutil.copy2(old_path, new_path)


folder_path = r"C:\Users\cheta\Downloads\0M in vivo airway Seg\0M in vivo airway Seg"
output = r"C:\Users\cheta\Downloads\0M in vivo airway Seg\data"
rename_files_with_bin(folder_path, output)

# tif to nii conversion

In [ ]:
import os
import numpy as np
import SimpleITK as sitk

# Input and output directories
input_folder = r'D:\Lung\Airways\mon_raw'
output_folder = r'D:\Lung\Airways\nii'

# Get a list of TIFF files in the input folder
tif_files = [f for f in os.listdir(input_folder) if f.endswith('.tif')]

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Iterate through TIFF files and convert them to NIfTI
for tif_file in tif_files:
    tif_path = os.path.join(input_folder, tif_file)
    output_name = tif_file[:-3] + 'nii'  # Output NIfTI filename
    output_path = os.path.join(output_folder, output_name)

    # Load TIFF image using SimpleITK
    tif_image = sitk.ReadImage(tif_path)

    # Convert SimpleITK image to NumPy array
    tif_array = sitk.GetArrayFromImage(tif_image)

    # Create a SimpleITK image from the NumPy array
    nii_image = sitk.GetImageFromArray(tif_array)

    # Save NIfTI image
    sitk.WriteImage(nii_image, output_path)

    print(f"Converted {tif_file} to {output_name}")

print("Conversion complete.")

# Slice and resize

In [ ]:
import os
import glob
import nibabel as nib
from skimage.transform import resize

def slice_and_resize_nifti(nifti_path, output_dir):
    # Load the NIfTI image
    img = nib.load(nifti_path)
    data = img.get_fdata()

    # Iterate over each slice
    for slice_idx in range(data.shape[2]):
        # Extract the slice
        slice_data = data[:, :, slice_idx]

        # Resize the slice to 256x256
        resized_slice = resize(slice_data, (256,256), preserve_range=True)

        # Generate the output filename
        #filename = os.path.splitext(os.path.basename(nifti_path))[0]
        #print(filename)
        slice_filename = nifti_filename[:-6] + '_'+str(slice_idx)+ "_M.nii"
        print(slice_filename)
        print(slice_filename)
        output_path = os.path.join(output_dir, slice_filename)

        # Save the resized slice
        nib.save(nib.Nifti1Image(resized_slice, img.affine), output_path)
        print(f"Saved {output_path}")

# Example usage
input_dir =r'D:\Lung\Airways\nii'
output_dir = r'D:\Lung\Airways\sliced'

nifti_files = [file for file in os.listdir(input_dir) if file.endswith("M.nii")]

# Process each NIfTI file
for nifti_filename in nifti_files:
    nifti_path = os.path.join(input_dir, nifti_filename)
    slice_and_resize_nifti(nifti_path, output_dir)

# Remove redundant slices (input:nii)

In [ ]:
import os
import shutil
import numpy as np
import nibabel as nib

# Input and output folder paths
input_folder = r'D:\Lung\Airways\sliced'
output_folder = r'D:\Lung\Airways\used'

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Iterate through each file in the input folder
for file_name in os.listdir(input_folder):
    if file_name.endswith("M.nii"):
        image_path = os.path.join(input_folder, file_name)
        mask_name = file_name.replace("_M.nii", "_K.nii")
        mask_path = os.path.join(input_folder, mask_name)
        
        # Load the mask and check the condition np.max > 1
        mask_data = nib.load(mask_path).get_fdata()
        if np.max(mask_data) >0:
            # Copy the image and mask files to the output folder
            shutil.copy2(image_path, output_folder)
            shutil.copy2(mask_path, output_folder)

# Rename based on slice index

In [ ]:
import os
import os

# Specify the directory path where your files are located
directory = r'D:\Lung\Airways\used'
path = r'D:\Lung\Airways\used1'

slice_mapping = {}
new_slice_index = 0

# Iterate over each file in the directory
for filename in os.listdir(directory):
    if filename.endswith("M.tif"):  # Filter only .nii files
        # Split the filename into different parts
        parts = filename.split('_')
        print(parts)


        # Extract the slice number from the filename
        slice_number = int(parts[2])
        print(slice_number)
        # Check if the slice number is already assigned a new index
        if slice_number not in slice_mapping:
            # Assign a new slice index to the current slice number
            slice_mapping[slice_number] = new_slice_index
            new_slice_index += 1

        # Get the new slice index for the current slice number
        new_slice_index = slice_mapping[slice_number]

        # Create the new filename with the desired naming convention
        new_filename = f"{parts[0]}_{parts[1]}_{new_slice_index}_{parts[3]}"

        # Check if the new filename already exists in the destination directory
        new_filepath = os.path.join(path, new_filename)
        counter = 1
        while os.path.exists(new_filepath):
            # Append a counter to make the filename unique
            new_filename = f"{parts[0]}_{parts[1]}_{new_slice_index}_{counter}_{parts[3]}"
            new_filepath = os.path.join(path, new_filename)
            counter += 1

        # Rename the file and move it to the new directory
        #shutil.copy2(os.path.join(directory, filename), new_filepath)

        print(f"Renamed {filename} to {new_filename}")

print("File renaming completed.")

# Binarize and NPY Conversion 

In [ ]:
import numpy as np
import tifffile as tiff
import os

folder_path = r'D:\Lung\Airways\used'
results_path = r'D:\Lung\Airways\npy_mon'

for filename in os.listdir(folder_path):
    if filename.endswith("M.nii"):
        # Load the 3D tiff image
        image_path = os.path.join(folder_path, filename)
        #image = tiff.imread(image_path)
        image = nib.load(image_path)
        image = image.get_fdata()
        #image = (image * 255).astype(np.uint8)
        #image = np.where(image>1,255,image)
        #image = np.where(image>1,1,image)
        print(image.max())
        print(image.min())
        #print(image.shape)
        # Save the numpy array as a .npy file
        name = f"{filename[:-3]}" + 'npy' 
        np.save(os.path.join(results_path, name), image)

# Preprocessing1

In [ ]:
import os
import numpy as np
import nibabel as nib
from skimage import exposure, filters
from skimage.transform import resize
from skimage.util import pad

def normalize_intensity(image):
    # Normalize intensity to [0, 1] range
    min_val = np.min(image)
    max_val = np.max(image)
    normalized_image = (image - min_val) / (max_val - min_val)
    return normalized_image

def histogram_equalization(image):
    # Apply histogram equalization to enhance contrast
    equalized_image = exposure.equalize_hist(image)
    return equalized_image

def apply_clahe(image, clip_limit=0.01, tile_grid_size=(8, 8)):
    # Apply Contrast Limited Adaptive Histogram Equalization (CLAHE)
    clahe = exposure.equalize_adapthist(image, clip_limit=clip_limit, kernel_size=tile_grid_size)
    return clahe

def denoise_image(image, sigma):
    # Apply Gaussian smoothing for denoising
    denoised_image = filters.gaussian(image, sigma=sigma)
    return denoised_image

def resize_image(image, target_shape):
    # Resize image to target shape
    resized_image = resize(image, target_shape, order=1, mode='constant', anti_aliasing=True)
    return resized_image

def pad_image(image, target_shape):
    # Pad image to target shape
    padded_image = pad(image, [(0, target_shape[i] - image.shape[i]) for i in range(image.ndim)], mode='constant')
    return padded_image

def preprocess_nifti(nifti_path, output_dir, target_shape=(128, 128, 128), sigma=1.0, apply_clahe=False):
    img = nib.load(nifti_path)
    data = img.get_fdata()

    # Preprocessing steps
    normalized_data = normalize_intensity(data)
    equalized_data = histogram_equalization(normalized_data)

    if apply_clahe:
        clahe_data = apply_clahe(equalized_data)
        preprocessed_data = denoise_image(clahe_data, sigma)
    else:
        preprocessed_data = denoise_image(equalized_data, sigma)

    resized_data = resize_image(preprocessed_data, target_shape)
    padded_data = pad_image(resized_data, target_shape)

    output_path = os.path.join(output_dir, os.path.basename(nifti_path))
    np.save(output_path, padded_data)

# usage
input_dir = r"D:\Input_Nifti_Files"
output_dir = r"D:\Preprocessed_Numpy_Files"

os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(input_dir):
    if filename.endswith(".nii") or filename.endswith(".nii.gz"):
        input_path = os.path.join(input_dir, filename)
        preprocess_nifti(input_path, output_dir)


# Preprocessing2

In [ ]:
import os
import numpy as np
import nibabel as nib
from scipy.ndimage import median_filter

# Function to apply contrast enhancement
def apply_contrast_filter(data):
    # Adjust the contrast here (you can modify this)
    contrast_factor = 1.5
    return data * contrast_factor

# Function to apply median filter
def apply_median_filter(data):
    # Set the filter size (you can modify this)
    filter_size = 3
    return median_filter(data, size=filter_size)

# Input and output directories
input_folder = r"D:\Preprocessed_Numpy_Files"
output_folder = r"D:\Preprocessed_Numpy_Files1"

# Create the output directory if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# List all NIfTI files in the input folder
nii_files = [file for file in os.listdir(input_folder) if file.endswith(".nii")]

# Loop through each NIfTI file
for nii_file in nii_files:
    input_path = os.path.join(input_folder, nii_file)
    output_path = os.path.join(output_folder, nii_file)

    # Load the NIfTI image
    img = nib.load(input_path)
    data = img.get_fdata()

    # Apply contrast enhancement
    contrast_filtered_data = apply_contrast_filter(data)

    # Apply median filter
    median_filtered_data = apply_median_filter(contrast_filtered_data)

    # Create a new NIfTI image with the filtered data
    filtered_img = nib.Nifti1Image(median_filtered_data, img.affine)

    # Save the filtered NIfTI image
    nib.save(filtered_img, output_path)

    print(f"Filtered and saved: {output_path}")

print("Filtering complete.")
